In [1]:
import pandas as pd
import requests
import numpy as np

In [3]:
df = pd.read_csv("graph.csv")

In [9]:
df.iloc[0]['periodStartUnix']

1620248400.0

In [12]:
# GraphQL endpoint for the Subgraph
endpoint = "https://api.thegraph.com/subgraphs/name/your-subgraph"

def query_avg_gas_price(start, end):
    query = """
    {
      transactions(where: { timestamp_gte: "START_TIMESTAMP", 
                            timestamp_lte: "END_TIMESTAMP",
                            pool_address: '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',
                          }) 
      {
        gasUsed
        gasPrice
      }
    }
    """.replace("START_TIMESTAMP", str(start)).replace("END_TIMESTAMP", str(end))

    response = requests.post(endpoint, json={'query': query})
    data = response.json()

    # Extract gas prices and calculate average
    gas_prices = [int(tx['gasPrice']) for tx in data['data']['transactions']]
    return np.mean(gas_prices) if gas_prices else 0

# Add a new column for average gas price
df['avgGasPrice'] = 0

for index, row in df.iterrows():
    avg_gas_price = query_avg_gas_price(row['periodStartUnix'], row['periodStartUnix']+3600)
    df.at[index, 'avgGasPrice'] = avg_gas_price

KeyError: 'data'

In [14]:
query = """
    {
      transactions(where: { timestamp_gte: '1620248400', 
                            timestamp_lte: '1620252000',
                            id: '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',
                          }) 
      {
        gasUsed
        gasPrice
      }
    }
    """

response = requests.post(endpoint, json={'query': query})
data = response.json()
data

{'errors': [{'locations': [{'line': 5, 'column': 43}],
   'message': "Unexpected `unexpected character '\\''`\nExpected `IntValue`, `FloatValue`, `StringValue`, `BlockString`, `true`, `false`, `null` or `Name`"}]}

In [15]:
import degenbot

In [19]:
lp = degenbot.V3LiquidityPool('0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')

ImportError: cannot import name 'Contract' from 'brownie' (/opt/anaconda3/envs/py10/lib/python3.10/site-packages/brownie/__init__.py)